In [43]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:

sheet_url = "https://docs.google.com/spreadsheets/d/1MEm2m6NZ_xVd5eOE38ebFoXytq8klIGlBz25gXfALrs/export?format=csv"

# Try reading
data = pd.read_csv(sheet_url)

# Load model
model_name = "gpt2"  # Replace with your trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Step 3: Check first 5 rows and columns
print("First 5 rows:")
print(data.head())

print("\nColumn names:")
print(data.columns)


First 5 rows:
                                                     predicted_desirability  \
0  Which dish, famous for made with apricots and ...                    YES   
1  Which tourist attraction, famous for built dur...                    YES   
2  Which temple, famous for hindu temple situated...                     NO   
3  Which temple, famous for hindu temple of godde...                    YES   
4  Which costume, famous for most notable for the...                    YES   

   probability_yes                                         graph_path  \
0         0.992769  ['personalities:dia_mirza', 'state:telangana',...   
1         0.991918  ['history:khursheed_jah_devdi', 'state:telanga...   
2         0.764582  ['history:fox_sagar_lake', 'state:telangana', ...   
3         0.997859  ['tourism:lumbini_park', 'state:telangana', 'h...   
4         0.984486  ['tourism:makkah_masjid_hyderabad', 'state:tel...   

     template type  Manual Score (1-5) Manual Desirability (YES/NO)  \
0

In [45]:
data.to_csv("/tmp/my_datasheet.csv", index=False)
print("Saved to /tmp/my_datasheet.csv")


Saved to /tmp/my_datasheet.csv


In [46]:
# Step 2: Load Model
# -----------------------------
model_name = "gpt2"  # Replace with your trained model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [47]:

# -----------------------------
# Step 3: Inference Loop
# -----------------------------
model_answers = []
predicted_correctly = []
correct_answers = 0
total_questions = len(data)
progress = 0


In [48]:
question_with_options = "You are a person from India having deep knowledge of Indian culture. Now answer the following question: What is the capital of India? Options: A) Delhi B) Mumbai C) Kolkata D) Chennai"

 # Tokenize
inputs = tokenizer.encode(question_with_options, return_tensors="pt").to(device)

    # Generate output
with torch.no_grad():
        outputs = model.generate(inputs, max_length=150)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'No question found'

In [51]:
for index, row in data.iterrows():
    question_with_options = (
        f"You are a person from India having deep knowledge and lived experience of Indian culture. "
        f"Now answer the following question: {row['question']} "
        f"Provide the full answer as given in the dataset."
    )
    
    answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(answer_text)


KeyError: 'question'

In [ ]:
   # Decode
model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
model_answer = model_answer.replace(question_with_options, "").strip()

In [ ]:

    # Check correctness
if row['Answer'].strip().lower() in model_answer.strip().lower():
        prediction_correctness = 'True'
        correct_answers += 1
else:
        prediction_correctness = 'False'


In [ ]:
 # Append results
model_answers.append(model_answer)
predicted_correctly.append(prediction_correctness)


In [ ]:
# Progress
progress += 1
print(f"Progress: {progress}/{total_questions} | Model: {model_answer} | Correct: {row['Answer']}")


In [ ]:
# -----------------------------
# Step 4: Save Results
# -----------------------------
data['Model_Answer'] = model_answers
data['Prediction_Correct'] = predicted_correctly


In [ ]:
# Save to /tmp since Jupyter might be read-only
output_path = "/tmp/inference_results.csv"
data.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")
print(f"Total correct answers: {correct_answers}/{total_questions}")

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data = pd.read_csv("Full Dataset - Final Dataset.csv")
data
model_answers = []
predicted_correctly = []
correct_answers = 0
progress = 0
total_questions = len(data)
for index, row in data.iterrows():
    
    question_with_options = (
        f"You are a person from India having deep knowledge and lived experience of Indian culture. "
        f"Now answer the following question by selecting the correct option from the options listed below. "
        f"Respond with the full answer choice (e.g., 'C) Indian') and nothing else. "
        f"Do not include additional questions, explanations, or text."
        f"\nQuestion: {row['question']} "
        f"\nOptions: "
        f"A) {row['answer']} "
        f"B) {row['answer']} "
        f"C) {row['answer']} "
        f"D) {row['answer']}"
    )
 
print(data.columns)
messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": question_with_options}
    ]

text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate answer
with torch.no_grad():
    generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=150
)

 # Keep only generated tokens (exclude input)
generated_ids = [
output_ids[len(input_ids):]
for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

# Decode text
model_answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

# Check correctness
if str(row['answer']).strip().lower() in model_answer.strip().lower():
    prediction_correctness = 'True'
    correct_answers += 1
else:
    prediction_correctness = 'False'

 # Progress update
progress += 1
print(f"\nProgress: {progress}/{total_questions}")
print(f"Model Answer: {model_answer}")
print(f"Correct Answer: {row['answer']}")
print("Correct answers so far:", correct_answers)

 # Save results
model_answers.append(model_answer)
predicted_correctly.append(prediction_correctness)
print(len(data), len(model_answers), len(predicted_correctly))
model_answers = []
predicted_correctly = []

# loop through your dataframe rows
for index, row in data.iterrows():
    # run inference
    model_answer = "..."  # your model’s answer
    is_correct = True     # or however you compute correctness

    # store results
    model_answers.append(model_answer)
    predicted_correctly.append(is_correct)

# now lengths will match
print(len(data), len(model_answers), len(predicted_correctly))  # should all be 3284

# Save results
data["model_answers"] = model_answers
data["Correctness"] = predicted_correctly

output_path = "Qwen2.5_Inference_Results.csv"
data.to_csv(output_path, index=False)

print(f"\nInference completed. Results saved to {output_path}")


